In [1]:
import cv2
import mediapipe as mp
import pydirectinput

In [2]:
mp_drawing= mp.solutions.drawing_utils
mp_hands = mp.solutions.hands


In [6]:
import enum
import cv2
from cv2 import waitKey

cap = cv2.VideoCapture(1)
cap.set(3, 640)  # Lowered resolution to 320x240 for performance
cap.set(4, 480)

frame_count = 0
skip_n = 1  # Only process every 2nd frame


with mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
                                ) as hands:
    print("")
    pose=''

    while cap.isOpened():
        success, img = cap.read()
        img = cv2.flip(img, 1)   
        if not success:
            break

        height, width, _ = img.shape
   
        frame_count += 1

        # Only process pose on some frames for less lag
        if frame_count % skip_n == 0:
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            result = hands.process(img_rgb)
            img = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR)

            try:
                if result.multi_hand_landmarks and result.multi_handedness:
                    for hand_id, hand_landmarks in enumerate(result.multi_hand_landmarks):
                        mp_drawing.draw_landmarks(
                            img,  # your current frame
                            hand_landmarks,  # landmarks to draw
                            mp_hands.HAND_CONNECTIONS,  # draw hand skeleton connections
                            mp_drawing.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=2),  # style for landmarks
                            mp_drawing.DrawingSpec(color=(0,0,255), thickness=2)  # style for connections
                        )
                        handedness_dict=result.multi_handedness[hand_id].classification[0]
                        hand_label = handedness_dict.label
                        index_tip = hand_landmarks.landmark[8]
                        thumb_tip = hand_landmarks.landmark[4]
                        

                        index_x, index_y = int(index_tip.x * width), int(index_tip.y * height)
                        thumb_x, thumb_y = int(thumb_tip.x * width), int(thumb_tip.y * height)

                        distance = ((index_x - thumb_x)**2 + (index_y - thumb_y)**2)**0.5
                        if distance < 60:
                            if hand_label == 'Left':
                                pose = 'acc'
                                pydirectinput.keyDown('right')
                                pydirectinput.keyUp('left')
                                print("Left Index finger and thumb tip touched")
                            elif hand_label == 'Right':
                                pose = 'brk'
                                pydirectinput.keyDown('left')
                                pydirectinput.keyUp('right')  
                                print("Right Index finger and thumb tip touched")

            except Exception as e:
                print(e)
                pass

        # Always show the pose and center line, even if skipping frame
        cv2.putText(img, pose, (10, 50), cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 0), 2)
        cv2.imshow('cargame', img)

        if waitKey(1) == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()



Right Index finger and thumb tip touched
Right Index finger and thumb tip touched
Right Index finger and thumb tip touched
Right Index finger and thumb tip touched
Right Index finger and thumb tip touched
Right Index finger and thumb tip touched
Right Index finger and thumb tip touched
Left Index finger and thumb tip touched
Left Index finger and thumb tip touched
Left Index finger and thumb tip touched
Left Index finger and thumb tip touched
Left Index finger and thumb tip touched
Left Index finger and thumb tip touched
Left Index finger and thumb tip touched
Left Index finger and thumb tip touched
Left Index finger and thumb tip touched
Left Index finger and thumb tip touched
Left Index finger and thumb tip touched
Left Index finger and thumb tip touched
Left Index finger and thumb tip touched
Left Index finger and thumb tip touched
Left Index finger and thumb tip touched
Left Index finger and thumb tip touched
Left Index finger and thumb tip touched
Left Index finger and thumb tip 